## Hello linear model
This tutorial is part of the Tensorflow getting started documentation at: https://www.tensorflow.org/tutorials/wide

## Convert data into a feature space
Extract a features of a sliding window from a given sample data.

In [99]:
#TODO: Change rotor label to the appropriate flight controls
import csv
from itertools import islice
import numpy as np

FEATURE_DATA_FILEPATH = './data/'

feature_csv_columns = ['average', 'variance']
imu_data_columns = ['gyro_roll', 'gyro_pitch', 'gyro_yaw', 'acc_x', 'acc_y', 'acc_z']

def feature_average(data_sample):
    fSum = 0;
    nIndex = 0;
    for data in data_sample:
        fSum = fSum + data
        nIndex = nIndex + 1
    return float(fSum / nIndex)

def feature_variance(data_sample):
    return np.var(data_sample)        

feature_calculations = {
    'average' : feature_average,
    'variance' : feature_variance
}

# Read a *.csv file and extract the sliding window
import collections
def extract_features_from_imu_data_samples_for_label(data_sample_filepath, features_filepath, data_label):
#     print('extracting for feature', data_label)
    with open(data_sample_filepath, 'r') as csv_file:
        # extract data records by row
        reader = csv.DictReader(csv_file)
        sliding_windows = []
        sliding_index = 0
        window_step_forward = 1
        window_length = 4
        
        # extract sliding windows from rows
        sliding_window_csv = []
        for row in reader:
            sliding_window_csv.append(row)
            if(len(sliding_window_csv) == window_length + 1):
                del sliding_window_csv[0]
            if(sliding_index % window_step_forward == 0 and len(sliding_window_csv) == window_length):
                sliding_windows.append(list(sliding_window_csv))
            sliding_index = sliding_index + 1
        running_window_lines = []
        
        for feature_name, feature_func in feature_calculations.items():
            for imu_data_column in imu_data_columns:
                window_sequences = []
                for sliding_window in sliding_windows:
                    window_sequence = []
                    for window in sliding_window:
                        window_sequence.append(int(window[imu_data_column]))
                    window_sequences.append(window_sequence)
#                 print(imu_data_column, ' - ', window_sequences)
                
                window_index = 0
                comma_index = 0;
                window_count = len(window_sequences)
                while len(running_window_lines) < window_count:
                    running_window_lines.append('')
                for window in window_sequences:
                    running_window_lines[window_index % window_count] += (str(feature_func(window))) + ','
#                     print(imu_data_column, ' _ ', feature_name, feature_func(window))
                    window_index = window_index + 1
#     print('PRINT FOR WINDOW ', window_count , running_window_lines)

    with open(features_filepath, 'a') as features_file:
        for feature_line in running_window_lines:
            features_file.write(data_label + ',' + feature_line[:-1] + '\n')
            print(feature_line[:-1])

            

# Iterate through the raw IMU data directories and get their labels
import glob
import os
labels = set()
print('Extracting labels for samples in raw data directory:')
for raw_data_dir in glob.glob('./data/**/raw/*', recursive=True):
    labels.add(os.path.basename(raw_data_dir))
print(labels)

#os.remove('./data/features.csv')
with open('./data/features.csv', 'w') as file:
    running_line = 'label,'
    for feature_columns in feature_csv_columns:
        for imu_data_column in imu_data_columns:
            running_line += feature_columns + '_' + imu_data_column + ','
    running_line = running_line[:-1]
    running_line += '\n'
    file.write(running_line)


# Iterate through each raw IMU data sample and extract their features:
for labelled_features in labels:
    print('Extracting features for:', labelled_features)
    for raw_data_dir in glob.glob('./data/**/raw/' + labelled_features + '/*.csv', recursive=False):
        extract_features_from_imu_data_samples_for_label(raw_data_dir, './data/features.csv', labelled_features)

Extracting labels for samples in raw data directory:
{'rotor3', 'rotor4', 'rotor2', 'rotor1'}
Extracting features for: rotor3
1117.25,-2123.75,-897.75,-176.0,607.0,-7462.25,10102859.1875,1848732.6875,78481.6875,243693.0,942106.5,29511798.1875
2207.0,-2420.75,-802.25,-220.0,104.25,-4587.75,3251328.5,744338.1875,52123.6875,206293.0,925138.6875,7044706.1875
3307.5,-1230.25,-208.5,-366.25,-839.0,-3286.75,6772180.25,1580654.6875,298730.75,48103.1875,1309934.0,8129025.6875
1811.5,-1704.75,-496.75,-416.75,-212.5,-4720.25,5101148.25,330347.1875,137238.6875,20883.6875,96716.75,1866064.1875
524.25,-1322.5,-635.25,-452.0,253.0,-6313.5,2556223.1875,127042.25,4679.6875,203594.0,561986.0,9360586.25
910.0,-1371.5,-702.0,-190.25,-183.0,-4859.25,886986.5,86764.25,7066.0,4991.1875,9138.0,614363.1875
4508.0,-2217.0,-291.25,-1157.5,-1549.5,-2107.0,19543408.5,200676.5,400233.6875,2605269.25,2966940.75,23179858.0
1327.75,-2294.5,-721.5,-284.0,-499.75,-5105.75,2054418.6875,124765.25,42373.25,131080.5,305037.


## Implement a linear classifier

In [ ]:
import tensorflow as tf
import os
import shutil
import sys

model_dir='./tmp/model'
train_data='./sample-data/rotor-tap-sample-data.csv'

# delete the model directory
shutil.rmtree(model_dir, ignore_errors=True)

# declare feature columns within csv
median_gyro_roll = tf.feature_column.numeric_column(key='median-gyro-roll', dtype=tf.float64);
median_gyro_pitch = tf.feature_column.numeric_column(key='median-gyro-pitch', dtype=tf.float64);
median_gyro_yaw = tf.feature_column.numeric_column(key='median-gyro-yaw', dtype=tf.float64);

median_acc_x = tf.feature_column.numeric_column(key='median-acc-x', dtype=tf.float64);
median_acc_y = tf.feature_column.numeric_column(key='median-acc-y', dtype=tf.float64);
median_acc_z = tf.feature_column.numeric_column(key='median-acc-z', dtype=tf.float64);

mean_gyro_roll = tf.feature_column.numeric_column(key='mean-gyro-roll', dtype=tf.float64);
mean_gyro_pitch = tf.feature_column.numeric_column(key='mean-gyro-pitch', dtype=tf.float64);
mean_gyro_yaw = tf.feature_column.numeric_column(key='mean-gyro-yaw', dtype=tf.float64);

# stack feature columns into a single array
imu_window_feature_columns = [median_gyro_roll, median_gyro_pitch, median_gyro_yaw,
        median_acc_x, median_acc_y, median_acc_z,
        mean_gyro_roll, mean_gyro_pitch, mean_gyro_yaw]

run_config=tf.estimator.RunConfig().replace(
    session_config=tf.ConfigProto(device_count={'GPU': 0})
)

def input_fn(data_file):
    assert tf.gfile.Exists(data_file),('%s not found')
    records_default = [['0'], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0],
                       [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
    csv_columns = [
                    'rotor',
                    'median-gyro-roll','median-gyro-pitch','median-gyro-yaw',
                    'median-acc-x','median-acc-y','median-acc-z',
                    'mean-gyro-roll','mean-gyro-pitch','mean-gyro-yaw',
                    'mean-acc-x','mean-acc-y','mean-acc-z',
                    'skew-gyro-roll','skew-gyro-pitch','skew-gyro-yaw',
                    'skew-acc-x','skew-acc-y','skew-acc-z'
                  ]
    
    def parse_csv(value):
        print('PARSING:', data_file)
        columns = tf.decode_csv(value, records_default)
        features = dict(zip(csv_columns, columns))
        labels = features.pop('rotor')
        print('LABELS:', labels)
        return features, labels
    
    dataset = tf.data.TextLineDataset(data_file)
    dataset = dataset.shuffle(200)
    dataset = dataset.map(parse_csv, 4)
    dataset = dataset.batch(200)
    return dataset

# model = tf.estimator.LinearClassifier(
#     model_dir=model_dir,
#     feature_columns=columns,
#     config=run_config,
#     n_classes=4,
#     label_vocabulary=['1', '2', '3', '4']
# )
model = tf.estimator.DNNClassifier(
    model_dir=model_dir,
    feature_columns=imu_window_feature_columns,
    config=run_config,
    hidden_units=[100, 75, 50, 25],
    n_classes=4,
    label_vocabulary=['1', '2', '3', '4']
)



# Train and evaluate the model every `FLAGS.epochs_per_eval` epochs.
for n in range(40 // 2):
    model.train(input_fn=lambda: input_fn(
        train_data))

    results = model.evaluate(input_fn=lambda: input_fn(
        train_data))

    # Display evaluation metrics
    print('Results at epoch', (n + 1) * 2)
    print('-' * 60)





# model.train(input_fn=lambda: input_fn(
#     train_data))

# results = model.evaluate(input_fn=lambda:input_fn(
#     train_data
# ))